In [1]:
ls

blocks.csv                         get_predicted.py*
dashboard.py                       get_predicted_shebang_namemain.py
deploy.py                          old/
deploy_clean.py                    rf_test_model.pkl
dev.ipynb                          transactions.csv
df_predict.csv


In [2]:
import pandas as pd
import numpy as np
import pickle
import plotly.express as px
import os

In [3]:
# load data
df = pd.read_csv("blocks.csv")
df2 = pd.read_csv("transactions.csv")

In [4]:
df.head()

,block_timestamp,base_fee_per_gas
0,2021-11-26 21:32:09,116925004757
1,2021-11-26 21:31:58,121935382610
2,2021-11-26 21:31:38,108389042873
3,2021-11-26 21:31:36,123043231276
4,2021-11-26 21:31:28,118132043184


In [5]:
len(df)

250

In [6]:
len(pd.Series(df['block_timestamp']).unique())

250

In [7]:
df2.head()

,block_timestamp,max_priority_fee_per_gas,receipt_effective_gas_price
0,2021-11-29 03:14:08,0,110000000000
1,2021-11-29 03:14:08,2000000000,106975341111
2,2021-11-29 03:14:08,2000000000,106975341111
3,2021-11-29 03:14:08,2000000000,106975341111
4,2021-11-29 03:14:08,0,132000000000


## agg

In [8]:
df2_agg = df2[['block_timestamp', 'receipt_effective_gas_price', 'max_priority_fee_per_gas']].groupby('block_timestamp').agg(['mean', 'count'])
df2_agg.columns = df2_agg.columns.map('_'.join).str.strip('_')
df2_agg.tail()


,receipt_effective_gas_price_mean,receipt_effective_gas_price_count,max_priority_fee_per_gas_mean,max_priority_fee_per_gas_count
block_timestamp,,,,
2021-11-29 03:13:34,1.140581e+11,206,2.256228e+09,206
2021-11-29 03:13:40,1.247622e+11,77,2.492227e+09,77
2021-11-29 03:13:41,1.072463e+11,123,7.923588e+09,123
2021-11-29 03:13:43,1.104397e+11,313,6.178903e+09,313
2021-11-29 03:14:08,1.110649e+11,133,2.829828e+09,133


In [9]:
#df2_agg = pd.DataFrame(df2_agg).sort_values(by=['block_timestamp'], ascending=False)

In [10]:
len(df2_agg)

514

In [11]:
len(df)

250

## merge

In [12]:
df_merge=df.merge(right=df2_agg, how='inner', on='block_timestamp')
df_merge = pd.DataFrame(df_merge).sort_values(by='block_timestamp', ascending=True)

In [13]:
df_merge = df_merge[-200:]

In [14]:
df_merge.head()

,block_timestamp,base_fee_per_gas,receipt_effective_gas_price_mean,receipt_effective_gas_price_count,max_priority_fee_per_gas_mean,max_priority_fee_per_gas_count


In [15]:
len(df_merge)

0

In [16]:
cols = ['base_fee_per_gas', 'receipt_effective_gas_price_count', 'receipt_effective_gas_price_mean', 'max_priority_fee_per_gas_mean']
for col in cols:
    # Last 5 blocks
    df_merge[col+'_pct_chg_last_5'] = df_merge[col]/df_merge[col].shift(5)-1
    # 100 blocks ago to 5 blocks ago percentage changes
    df_merge[col+'_pct_chg_last_100_to_5'] = df_merge[col].shift(5)/df_merge[col].shift(100)-1


In [17]:
#df_merge = pd.DataFrame(df_merge).sort_values(by='block_timestamp', ascending=False)

In [18]:
df_merge = df_merge[-100:]
df_merge.head()

,block_timestamp,base_fee_per_gas,receipt_effective_gas_price_mean,receipt_effective_gas_price_count,max_priority_fee_per_gas_mean,max_priority_fee_per_gas_count,base_fee_per_gas_pct_chg_last_5,base_fee_per_gas_pct_chg_last_100_to_5,receipt_effective_gas_price_count_pct_chg_last_5,receipt_effective_gas_price_count_pct_chg_last_100_to_5,receipt_effective_gas_price_mean_pct_chg_last_5,receipt_effective_gas_price_mean_pct_chg_last_100_to_5,max_priority_fee_per_gas_mean_pct_chg_last_5,max_priority_fee_per_gas_mean_pct_chg_last_100_to_5


In [19]:
df_merge.tail()

,block_timestamp,base_fee_per_gas,receipt_effective_gas_price_mean,receipt_effective_gas_price_count,max_priority_fee_per_gas_mean,max_priority_fee_per_gas_count,base_fee_per_gas_pct_chg_last_5,base_fee_per_gas_pct_chg_last_100_to_5,receipt_effective_gas_price_count_pct_chg_last_5,receipt_effective_gas_price_count_pct_chg_last_100_to_5,receipt_effective_gas_price_mean_pct_chg_last_5,receipt_effective_gas_price_mean_pct_chg_last_100_to_5,max_priority_fee_per_gas_mean_pct_chg_last_5,max_priority_fee_per_gas_mean_pct_chg_last_100_to_5


In [20]:
len(df_merge)

0

## seasonality

In [21]:
df_merge.head()

,block_timestamp,base_fee_per_gas,receipt_effective_gas_price_mean,receipt_effective_gas_price_count,max_priority_fee_per_gas_mean,max_priority_fee_per_gas_count,base_fee_per_gas_pct_chg_last_5,base_fee_per_gas_pct_chg_last_100_to_5,receipt_effective_gas_price_count_pct_chg_last_5,receipt_effective_gas_price_count_pct_chg_last_100_to_5,receipt_effective_gas_price_mean_pct_chg_last_5,receipt_effective_gas_price_mean_pct_chg_last_100_to_5,max_priority_fee_per_gas_mean_pct_chg_last_5,max_priority_fee_per_gas_mean_pct_chg_last_100_to_5


In [22]:
df_merge['local_date'] = pd.to_datetime(df_merge['block_timestamp']).dt.tz_localize('utc').dt.tz_convert('US/Eastern')
df_merge['date'] = df_merge['local_date'].dt.date
df_merge['hour'] = df_merge['local_date'].dt.hour
df_merge['minute'] = df_merge['local_date'].dt.minute
df_merge['weekday'] = df_merge['local_date'].dt.weekday
df_merge.head()

,block_timestamp,base_fee_per_gas,receipt_effective_gas_price_mean,receipt_effective_gas_price_count,max_priority_fee_per_gas_mean,max_priority_fee_per_gas_count,base_fee_per_gas_pct_chg_last_5,base_fee_per_gas_pct_chg_last_100_to_5,receipt_effective_gas_price_count_pct_chg_last_5,receipt_effective_gas_price_count_pct_chg_last_100_to_5,receipt_effective_gas_price_mean_pct_chg_last_5,receipt_effective_gas_price_mean_pct_chg_last_100_to_5,max_priority_fee_per_gas_mean_pct_chg_last_5,max_priority_fee_per_gas_mean_pct_chg_last_100_to_5,local_date,date,hour,minute,weekday


In [23]:
df_merge['hour_dummy'] = 0
start_hour = 2
end_hour = 9
mask = (df_merge['hour'] < start_hour) | (df_merge['hour'] > end_hour)
df_merge.loc[mask, 'hour_dummy'] = 1

df_merge['minute_dummy'] = 0
start_minute = 1
end_minute = 6
mask = (df_merge['minute'] >= start_minute) & (df_merge['minute'] <= end_minute)
df_merge.loc[mask, 'minute_dummy'] = 1

df_merge['weekday_dummy'] = 0
start_weekday = 1
end_weekday = 4
mask = (df_merge['weekday'] >= start_weekday) & (df_merge['weekday'] <= end_weekday)
df_merge.loc[mask, 'weekday_dummy'] = 1

In [24]:
df_merge.head()

,block_timestamp,base_fee_per_gas,receipt_effective_gas_price_mean,receipt_effective_gas_price_count,max_priority_fee_per_gas_mean,max_priority_fee_per_gas_count,base_fee_per_gas_pct_chg_last_5,base_fee_per_gas_pct_chg_last_100_to_5,receipt_effective_gas_price_count_pct_chg_last_5,receipt_effective_gas_price_count_pct_chg_last_100_to_5,...,max_priority_fee_per_gas_mean_pct_chg_last_5,max_priority_fee_per_gas_mean_pct_chg_last_100_to_5,local_date,date,hour,minute,weekday,hour_dummy,minute_dummy,weekday_dummy


In [25]:
df_merge.columns

Index(['block_timestamp', 'base_fee_per_gas',
       'receipt_effective_gas_price_mean', 'receipt_effective_gas_price_count',
       'max_priority_fee_per_gas_mean', 'max_priority_fee_per_gas_count',
       'base_fee_per_gas_pct_chg_last_5',
       'base_fee_per_gas_pct_chg_last_100_to_5',
       'receipt_effective_gas_price_count_pct_chg_last_5',
       'receipt_effective_gas_price_count_pct_chg_last_100_to_5',
       'receipt_effective_gas_price_mean_pct_chg_last_5',
       'receipt_effective_gas_price_mean_pct_chg_last_100_to_5',
       'max_priority_fee_per_gas_mean_pct_chg_last_5',
       'max_priority_fee_per_gas_mean_pct_chg_last_100_to_5', 'local_date',
       'date', 'hour', 'minute', 'weekday', 'hour_dummy', 'minute_dummy',
       'weekday_dummy'],
      dtype='object')

##### FROM ROB: 
'''rf_features = ['base_fee_per_gas_pct_chg_last_100_to_5', 'base_fee_per_gas_pct_chg_last_5',
             'number_transactions_in_block_pct_chg_last_100_to_5', 'number_transactions_in_block_pct_chg_last_5',
             'effective_gas_price_mean_pct_chg_last_100_to_5', 'effective_gas_price_mean_pct_chg_last_5',
             'max_priority_fee_per_gas_mean_pct_chg_last_100_to_5', 'max_priority_fee_per_gas_mean_pct_chg_last_5',
             'minute_dummy', 'hour_dummy', 'weekday_dummy']
'''

In [26]:
df_merge.rename(columns={
       'receipt_effective_gas_price_count_pct_chg_last_5':'number_transactions_in_block_pct_chg_last_5',
       'receipt_effective_gas_price_count_pct_chg_last_100_to_5':'number_transactions_in_block_pct_chg_last_100_to_5',
       'receipt_effective_gas_price_mean_pct_chg_last_5':'effective_gas_price_mean_pct_chg_last_5',
       'receipt_effective_gas_price_mean_pct_chg_last_100_to_5':'effective_gas_price_mean_pct_chg_last_100_to_5',
        }, inplace=True)


In [27]:
df_merge.columns

Index(['block_timestamp', 'base_fee_per_gas',
       'receipt_effective_gas_price_mean', 'receipt_effective_gas_price_count',
       'max_priority_fee_per_gas_mean', 'max_priority_fee_per_gas_count',
       'base_fee_per_gas_pct_chg_last_5',
       'base_fee_per_gas_pct_chg_last_100_to_5',
       'number_transactions_in_block_pct_chg_last_5',
       'number_transactions_in_block_pct_chg_last_100_to_5',
       'effective_gas_price_mean_pct_chg_last_5',
       'effective_gas_price_mean_pct_chg_last_100_to_5',
       'max_priority_fee_per_gas_mean_pct_chg_last_5',
       'max_priority_fee_per_gas_mean_pct_chg_last_100_to_5', 'local_date',
       'date', 'hour', 'minute', 'weekday', 'hour_dummy', 'minute_dummy',
       'weekday_dummy'],
      dtype='object')

In [28]:
with open('rf_test_model.pkl', 'rb') as model:
    rf = pickle.load(model)

/Users/anthonybaldor/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/anthonybaldor/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.24.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [29]:
features = ['base_fee_per_gas_pct_chg_last_100_to_5', 
            'base_fee_per_gas_pct_chg_last_5',
            'number_transactions_in_block_pct_chg_last_100_to_5', 
            'number_transactions_in_block_pct_chg_last_5',
            'effective_gas_price_mean_pct_chg_last_100_to_5', 
            'effective_gas_price_mean_pct_chg_last_5',
            'max_priority_fee_per_gas_mean_pct_chg_last_100_to_5', 
            'max_priority_fee_per_gas_mean_pct_chg_last_5',
            'minute_dummy', 'hour_dummy', 'weekday_dummy']
df_predict = df_merge[features]



'''df_predict = df_merge[features].astype({
    'base_fee_per_gas_pct_chg_last_100_to_5': np.float32,
    'base_fee_per_gas_pct_chg_last_5': np.float32,
    'receipt_effective_gas_price_count_pct_chg_last_100_to_5': np.float32,
    'receipt_effective_gas_price_count_pct_chg_last_5': np.float32,
    'receipt_effective_gas_price_mean_pct_chg_last_100_to_5': np.float32,
    'receipt_effective_gas_price_mean_pct_chg_last_5': np.float32,
    'max_priority_fee_per_gas_mean_pct_chg_last_100_to_5': np.float32,
    'max_priority_fee_per_gas_mean_pct_chg_last_5': np.float32,
})
'''

In [30]:
df_predict.tail()

,base_fee_per_gas_pct_chg_last_100_to_5,base_fee_per_gas_pct_chg_last_5,number_transactions_in_block_pct_chg_last_100_to_5,number_transactions_in_block_pct_chg_last_5,effective_gas_price_mean_pct_chg_last_100_to_5,effective_gas_price_mean_pct_chg_last_5,max_priority_fee_per_gas_mean_pct_chg_last_100_to_5,max_priority_fee_per_gas_mean_pct_chg_last_5,minute_dummy,hour_dummy,weekday_dummy


In [31]:
mask100 = (df_predict['max_priority_fee_per_gas_mean_pct_chg_last_100_to_5'] >= 17.677177)
df_predict.loc[mask100, 'max_priority_fee_per_gas_mean_pct_chg_last_100_to_5'] = 17.677177
mask5 = (df_predict['max_priority_fee_per_gas_mean_pct_chg_last_5'] >= 10.24462)
df_predict.loc[mask5, 'max_priority_fee_per_gas_mean_pct_chg_last_5'] = 10.24462

In [32]:
#columns = df_predict.columns
#values = [0.431637, 0.309303, 8.809524, 8.9, 17.677177, 16.710033, 15.495496, 10.24462]
#for i in columns[:8]:
#    df_predict[i].replace(np.inf, values[columns.get_loc(i)], inplace=True)
#    #print(values[columns.get_loc(i)])
#    #print(df_predict[i])


In [33]:
predicted = rf.predict(df_predict)

ValueError: Found array with 0 sample(s) (shape=(0, 11)) while a minimum of 1 is required.

In [ ]:
print(predicted)

In [ ]:
print(len(predicted))
print(len(df_merge))

In [ ]:
chart = df_merge[['block_timestamp','receipt_effective_gas_price_mean']]

In [ ]:
chart['predicted'] = predicted

In [ ]:
chart.head()

In [ ]:
chart.tail()

In [ ]:
chart.set_index('block_timestamp')

In [ ]:
chart.set_index('block_timestamp').plot()

In [ ]:
chart.to_csv('chart.csv', index=False)


In [ ]:
dir_path = os.path.dirname(os.path.realpath(__file__))